In [61]:
import sys 
import os 
from dotenv import load_dotenv 
import dspy 
from IPython.display import Markdown as md , display 
# --- Load Env --- 
if not load_dotenv(dotenv_path=os.path.join(os.getcwd(), '.env')):
    raise ValueError(" this .env file not found")


In [62]:
# --- Check Version --- 
print(f"Python {sys.version.split()[0]}")
print(f"DSpy {dspy.__version__}")

Python 3.11.5
DSpy 3.0.3


In [63]:
# --- Configure Model --- 
gemini = dspy.LM(
    model="gemini/gemini-flash-latest",
    api_key=os.getenv("GEMINI_API_KEY")
)
dspy.configure(lm=gemini)
print("Gemini Model Configured")


Gemini Model Configured


In [64]:
# --- Define Signatures --- 
class ContextToScript(dspy.Signature):
    """
        Generate podcast Script from Context 
    """
    context = dspy.InputField(desc="Context for the podcast script")
    podcast_title = dspy.InputField(desc="Title of the podcast")
    script = dspy.OutputField(desc="Generated podcast script")


In [70]:
# --- Here We define Some Evals --- 
def evaluate_coherence(script: str) -> float:
    """
    Evaluate based on:
        - Non-empty lines from the script 
        - Speaker alternation pattern
        - Return alternations / total_transitions 
    """
    lines = [l.strip() for l in script.split("\n") if l.strip()]
    speaker_pattern = [l.startswith("Speaker") for l in lines]
    
    if len(speaker_pattern) < 2:
        return 0.0
    
    alternations = sum(1 for i in range(len(speaker_pattern)-1) 
                       if speaker_pattern[i] != speaker_pattern[i+1])
    total_transitions = len(speaker_pattern) - 1
    
    return alternations / total_transitions if total_transitions > 0 else 0.0


def evaluate_engagement(script: str) -> float:
    """
        Check for engaging elements like questions, exclamations.
    """
    engagement_markers = ['?', '!', 'fascinating', 'incredible', 
                          'interesting', 'amazing', 'Can you', 'Wow']
    count = sum(script.lower().count(marker.lower()) 
                for marker in engagement_markers)
    words = len(script.split())
    return min(count / max(words / 50, 1), 1.0)


def evaluate_structure(script: str) -> float:
    """
        Check if script has proper speaker labels.
    """
    has_speaker1 = 'Speaker 1' in script
    has_speaker2 = 'Speaker 2' in script
    has_multiple_turns = script.count('Speaker') >= 4
    
    score = sum([has_speaker1, has_speaker2, has_multiple_turns]) / 3
    return score


def script_quality_metric(gold, pred, trace=None) -> float:
    """
    Combined quality metric for podcast scripts.
    
    Weights: 
        Coherence: 0.4
        Engagement: 0.3
        Structure: 0.3
    """
    script_text = pred.script if hasattr(pred, 'script') else str(pred)
    
    coherence = evaluate_coherence(script_text)
    engagement = evaluate_engagement(script_text)
    structure = evaluate_structure(script_text)
    
    return 0.4 * coherence + 0.3 * engagement + 0.3 * structure


In [71]:
# --- Here We Define the DataSet --- 
training_examples = [
    # ---  Healthcare AI ----
    dspy.Example(
        context="AI in healthcare: Machine learning models now detect cancer with 95% accuracy in early stages. Deep learning algorithms analyze medical imaging faster than radiologists. AI-powered diagnostics reduce false negatives by 40%. Hospitals are implementing AI triage systems for emergency departments.",
        podcast_title="The Future of AI in Healthcare",
        script="Speaker 1: Today we're exploring how artificial intelligence is revolutionizing healthcare diagnostics.\nSpeaker 2: That's incredible! Can you break down some specific breakthroughs we're seeing?\nSpeaker 1: Absolutely. Machine learning models are now detecting cancer with 95% accuracy in early stages, which is truly remarkable.\nSpeaker 2: Wow! How does that compare to traditional methods?"
    ).with_inputs("context", "podcast_title"),
    
    #  ---  Climate Technology ---
    dspy.Example(
        context="Climate tech innovations: Carbon capture technology now removes CO2 at $100 per ton. Renewable energy storage solutions extend battery life to 20 years. Solar efficiency has reached 47% in lab settings. Green hydrogen production costs dropped 60% since 2020.",
        podcast_title="Climate Tech Breakthroughs",
        script="Speaker 1: Welcome to today's episode on climate technology innovations that are changing the game.\nSpeaker 2: I'm really excited about this topic! What's the most promising development you've seen?\nSpeaker 1: Well, carbon capture technology has hit a major milestone - it's now removing CO2 at just $100 per ton.\nSpeaker 2: That's amazing! What does this mean for fighting climate change?"
    ).with_inputs("context", "podcast_title"),
    
    #  ---  Blockchain & Web3 ---
    dspy.Example(
        context="Web3 evolution: Decentralized finance (DeFi) protocols now manage over $50 billion in assets. Layer-2 solutions reduced Ethereum gas fees by 90%. NFT technology expands beyond art into real estate and identity verification. Smart contracts enable trustless transactions without intermediaries.",
        podcast_title="Web3 and the Future of Finance",
        script="Speaker 1: Let's dive into the world of Web3 and how it's transforming traditional finance.\nSpeaker 2: Perfect timing! I keep hearing about DeFi but I'm not entirely clear on what makes it revolutionary.\nSpeaker 1: Great question. DeFi protocols are now managing over $50 billion in assets, completely removing traditional banking intermediaries.\nSpeaker 2: That's mind-blowing! But is it actually safe and secure?"
    ).with_inputs("context", "podcast_title"),
    
    #  ---  Space Exploration ---
    dspy.Example(
        context="Space exploration advances: SpaceX's Starship completed first orbital test flight. NASA's Artemis program plans permanent Moon base by 2030. Mars rover Perseverance discovered organic molecules suggesting past life. Private space stations are scheduled for 2025 launch.",
        podcast_title="The New Space Race",
        script="Speaker 1: Today we're talking about the incredible advances in space exploration that feel straight out of science fiction.\nSpeaker 2: I've been following SpaceX's Starship development closely. What's the latest?\nSpeaker 1: They've just completed their first successful orbital test flight, which is a massive achievement.\nSpeaker 2: Unbelievable! What does this mean for Mars colonization?"
    ).with_inputs("context", "podcast_title"),
    
    #  ---  Education Technology ---
    dspy.Example(
        context="EdTech transformation: AI tutors personalize learning for each student's pace and style. VR classrooms enable immersive history lessons. Adaptive learning platforms increase retention rates by 35%. Gamification boosts student engagement by 60% in STEM subjects.",
        podcast_title="Reimagining Education with Technology",
        script="Speaker 1: Welcome! Today we're exploring how technology is fundamentally changing the way we learn.\nSpeaker 2: This is fascinating. I've heard about AI tutors - how effective are they really?\nSpeaker 1: The data is impressive. AI tutoring systems are personalizing learning and boosting retention rates by 35%.\nSpeaker 2: That's remarkable! How does it adapt to different learning styles?"
    ).with_inputs("context", "podcast_title"),
    
    #  ---  Quantum Computing ---
    dspy.Example(
        context="Quantum computing progress: Google's quantum processor solved problems in 200 seconds that would take classical computers 10,000 years. IBM launched 433-qubit quantum computer. Quantum algorithms promise to revolutionize drug discovery and cryptography. Error correction breakthroughs make quantum systems more stable.",
        podcast_title="The Quantum Revolution",
        script="Speaker 1: Let's talk about quantum computing - a technology that sounds like pure magic.\nSpeaker 2: I've tried to understand it but it's so complex! Can you explain what makes it so powerful?\nSpeaker 1: Here's what blows my mind: Google's quantum processor solved in 200 seconds what would take classical computers 10,000 years.\nSpeaker 2: Wait, what? How is that even possible?"
    ).with_inputs("context", "podcast_title"),
    
    #  ---  Synthetic Biology ---
    dspy.Example(
        context="Synthetic biology advances: Scientists engineered bacteria to produce spider silk proteins. CRISPR gene editing approved for treating sickle cell disease. Lab-grown meat achieves cost parity with traditional meat. Synthetic cells created that can perform photosynthesis more efficiently than plants.",
        podcast_title="Engineering Life: Synthetic Biology",
        script="Speaker 1: Today's topic is synthetic biology - where we're literally programming living organisms.\nSpeaker 2: This sounds like something from a sci-fi movie! What's the most exciting development?\nSpeaker 1: Scientists have engineered bacteria to produce spider silk proteins, which could revolutionize materials science.\nSpeaker 2: That's incredible! What are the practical applications of that?"
    ).with_inputs("context", "podcast_title"),
    
    #  ---  Autonomous Vehicles ---
    dspy.Example(
        context="Self-driving technology: Waymo autonomous taxis now serve 100,000 rides per week in San Francisco. Tesla's Full Self-Driving beta shows 85% reduction in accidents. Delivery robots operate in 50+ cities worldwide. Autonomous trucks completed first cross-country freight delivery.",
        podcast_title="The Road to Autonomous Transportation",
        script="Speaker 1: Welcome to today's discussion on autonomous vehicles and how they're reshaping transportation.\nSpeaker 2: I'm both excited and nervous about self-driving cars. Where are we right now?\nSpeaker 1: We're further along than most people realize. Waymo is now serving 100,000 autonomous taxi rides per week in San Francisco.\nSpeaker 2: Seriously? Are they actually safer than human drivers?"
    ).with_inputs("context", "podcast_title"),
    
    #  ---  Neuromorphic Computing ---
    dspy.Example(
        context="Brain-inspired computing: Intel's neuromorphic chip processes data 1000x more energy-efficiently than traditional processors. Neuromorphic systems enable real-time edge AI processing. Brain-computer interfaces achieve 90% accuracy in translating thoughts to text. Neuromorphic sensors mimic human vision and hearing.",
        podcast_title="Computers That Think Like Brains",
        script="Speaker 1: Let's explore neuromorphic computing - technology that mimics how our brains actually work.\nSpeaker 2: This sounds fascinating! How is it different from regular computer chips?\nSpeaker 1: Intel's neuromorphic chip processes data 1000 times more energy-efficiently than traditional processors by mimicking neural networks.\nSpeaker 2: That's amazing! What real-world applications does this enable?"
    ).with_inputs("context", "podcast_title"),
]

In [72]:
# --- Here We Apply the Optimization --- 
class PodCastScriptGenerator(dspy.Module):
    def __init__():
        super().__init__()
        self.context_to_script = dspy.ChainOfThought
    def forward(self,context,podcast_title):
        return self.context_to_script(context,podcast_title=podcast_title)
# --- Optimize ---
script_generator = PodcastScriptGenerator()
teleprompter = dspy.BootstrapFewShot(
    metric=script_quality_metric,
    max_bootstrapped_demos=9
)
try:
    optimized_generator = teleprompter.compile(
        script_generator,
        trainset=training_examples
    )
    print("Optimization Complete ")
except Exception as e:
    print(f"Optimization failed: {e}")


100%|██████████| 9/9 [00:00<00:00, 20.36it/s]

Bootstrapped 9 full traces after 8 examples for up to 1 rounds, amounting to 9 attempts.
Optimization Complete 


In [74]:
test_context = """
LangGraph enables building stateful AI agents with checkpointing and 
human-in-the-loop workflows. It provides a framework for creating 
complex multi-step agent systems.
"""

baseline = script_generator(context=test_context, podcast_title="Agentic AI")
optimized = optimized_generator(context=test_context, podcast_title="Agentic AI")

display(md(f"### Baseline Quality: {script_quality_metric(None, baseline):.2f}"))
display(md(f"```\n{baseline.script[:800]}...\n```"))

display(md(f"### Optimized Quality: {script_quality_metric(None, optimized):.2f}"))
display(md(f"```\n{optimized.script[:800]}...\n```"))

### Baseline Quality: 0.00

```
**(Intro Music fades)**

**Host (Alex):** Welcome back to Agentic AI, the podcast exploring the cutting edge of autonomous systems. Today, we’re talking about the architecture that makes complex AI agents not just smart, but reliable: LangGraph.

When we talk about advanced AI agents—the ones that plan, execute, and iterate—we’re quickly moving beyond simple, single-turn prompts. We’re building systems that require memory, decision-making, and the ability to handle failure gracefully.

This is precisely the challenge LangGraph was built to solve. It provides a powerful framework for creating complex, multi-step agent systems by modeling the workflow as a graph of nodes and edges.

The first critical feature is **statefulness**. Unlike stateless systems, LangGraph allows your agent to maint...
```

### Optimized Quality: 0.60

```
Speaker 1: Welcome to 'Agentic AI.' Today, we're talking about the next evolution of AI—systems that can plan, execute, and manage complex, multi-step tasks.
Speaker 2: That sounds like a huge leap from simple chatbots. If these agents are running complex workflows, how do developers manage all that state and ensure reliability?
Speaker 1: That's where frameworks like LangGraph come in. It provides the essential structure for creating these complex multi-step agent systems. It makes the agents stateful, meaning they remember where they are in a long process.
Speaker 2: So, if a task fails halfway through, they don't have to start over?
Speaker 1: Exactly. LangGraph enables checkpointing. The agent can save its progress, making the entire system far more robust and reliable.
Speaker 2: And ...
```